<a href="https://colab.research.google.com/github/zanqi/tinybert/blob/main/bert_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Getting Data

In [1]:
# We always start with a dataset to train on. Let's download the MIND dataset
!wget https://raw.githubusercontent.com/zanqi/tinybert/main/news.tsv.zip


--2023-07-13 01:45:48--  https://raw.githubusercontent.com/zanqi/tinybert/main/news.tsv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10694395 (10M) [application/zip]
Saving to: ‘news.tsv.zip’

news.tsv.zip        100%[===================>]  10.20M  --.-KB/s    in 0.09s   

2023-07-13 01:45:48 (115 MB/s) - ‘news.tsv.zip’ saved [10694395/10694395]



In [2]:
! unzip {'news.tsv.zip'}

Archive:  news.tsv.zip
  inflating: news.tsv                


In [3]:
! pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00


In [4]:
import pandas as pd

data = pd.read_csv('news.tsv',header=None,sep='\t')

In [5]:
data.columns=['News ID',
"Category",
"SubCategory",
"Title",
"Abstract",
"URL",
"Title Entities",
"Abstract Entities "]

text_data = data.iloc[:, 3:5]
text_data.Abstract.str.len().describe()
# min(abstract), max(abstract)

count    48616.000000
mean       216.089682
std        154.471979
min          1.000000
25%        100.000000
50%        149.000000
75%        394.000000
max       2603.000000
Name: Abstract, dtype: float64

In [6]:
text_data.iloc[text_data.Abstract.str.len().argmax()].Abstract

'Real talk. Demi Moore got candid about a variety of topics in her new book, Inside Out, including her famous exes, substance abuse struggles and her heartbreaking sexual assault. "The same question kept going through my head: How did I get here?" the 56-year-old actress began in the memoir, which was released on Tuesday, September 24. "The husband who I\'d thought was the love of my life had cheated on me and then decided he didn\'t want to work on our marriage. My children weren\'t speaking me. … Is this life? I wondered. Because if this is it, I\'m done." Moore provided insight into all three of her marriages in the book. She was married to Freddy Moore from 1980 to 1985, Bruce Willis from 1987 to 2000 and Ashton Kutcher from 2005 to 2013. The end of the G.I. Jane star\'s relationship with the former That 70\'s Show star, however, seemed to have the biggest impact on her. "I lost me," the Ghost actress told Diane Sawyer on Good Morning America on Monday, September 23, about their sp

In [7]:
text_data[text_data.Abstract.str.len() > 10]

,Title,Abstract
0,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."
...,...,...
51275,Realme takes chunk of India mobile market as S...,Over 400 percent more phones shipped year-on-year
51276,Young Northeast Florida fans flock to U.S. wom...,When the U.S. women's national soccer team arr...
51277,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...
51279,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b..."


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
sent_text = nltk.sent_tokenize('Real talk. Demi Moore got candid about a variety of topics in her new book, Inside Out, including her famous exes, substance abuse struggles and her heartbreaking sexual assault. "The same question kept going through my head: How did I get here?" the 56-year-old actress began in the memoir, which was released on Tuesday, September 24.')
sent_text

['Real talk.',
 'Demi Moore got candid about a variety of topics in her new book, Inside Out, including her famous exes, substance abuse struggles and her heartbreaking sexual assault.',
 '"The same question kept going through my head: How did I get here?"',
 'the 56-year-old actress began in the memoir, which was released on Tuesday, September 24.']

In [10]:
sents = [nltk.sent_tokenize(a) for a in text_data[text_data.Abstract.str.len() > 10].Abstract]
abstracts = [a for a in sents if len(a) > 1]
len(abstracts), abstracts[:10]

(21962,
 [['Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine.',
   'Next to him was an empty helmet propped up to trick snipers, already perforated with multiple holes.'],
  ["I felt like I was a fraud, and being an NBA wife didn't help that.",
   'In fact, it nearly destroyed me.'],
  ["They seem harmless, but there's a very good reason you shouldn't ignore them.",
   "The post How to Get Rid of Skin Tags, According to a Dermatologist appeared first on Reader's Digest."],
  ['Several fines came down against NFL players for criticizing officiating this week.',
   "It's a very bad look for the league."],
  ['When there are active closings, view them here.',
   'WXII 12 News receives a number of phone calls and e-mails from viewers with questions.',
   'Sign up for our Newsletters To report a closure, please visit wxii.reportclosing.com The weather closing system is a viewer-operated system.',
   'Employees of WXII-TV and WXII12.com DO NOT en

In [11]:
print("length of dataset in sentences: ", sum(len(a) for a in abstracts))

length of dataset in sentences:  74948


In [12]:
zipped = [zip(a, a[1:]) for a in abstracts]
pairs = [(s1, s2) for z in zipped for s1, s2 in z]
len(pairs)

52986

In [13]:
n = int(0.9*len(pairs)) # first 90% will be train, rest val
train_data = pairs[:n]
val_data = pairs[n:]
train_data[:10]

[('Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine.',
  'Next to him was an empty helmet propped up to trick snipers, already perforated with multiple holes.'),
 ("I felt like I was a fraud, and being an NBA wife didn't help that.",
  'In fact, it nearly destroyed me.'),
 ("They seem harmless, but there's a very good reason you shouldn't ignore them.",
  "The post How to Get Rid of Skin Tags, According to a Dermatologist appeared first on Reader's Digest."),
 ('Several fines came down against NFL players for criticizing officiating this week.',
  "It's a very bad look for the league."),
 ('When there are active closings, view them here.',
  'WXII 12 News receives a number of phone calls and e-mails from viewers with questions.'),
 ('WXII 12 News receives a number of phone calls and e-mails from viewers with questions.',
  'Sign up for our Newsletters To report a closure, please visit wxii.reportclosing.com The weather closing system is a v

In [14]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [15]:
tokens = [tokenizer(s)['input_ids'] for a in abstracts[:1] for s in a]

In [16]:
encoded = tokenizer(abstracts[0][0])
print(encoded) # len(token_ids), tokenizer.decode(token_ids)
tokenizer.decode(encoded['input_ids'])


{'input_ids': [101, 8318, 1012, 7332, 9587, 29358, 7231, 3215, 18652, 2058, 1037, 27372, 1997, 5472, 8641, 2012, 1996, 2392, 2240, 1997, 1996, 2162, 1999, 5924, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


'[CLS] lt. ivan molchanets peeked over a parapet of sand bags at the front line of the war in ukraine. [SEP]'

In [17]:
# tokenizer.vocab.items()

In [18]:
import torch
import random
torch.manual_seed(1337)

batch_size = 4

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data), (batch_size,))
  next_ix =[i if random.random() < 0.5 else (i+1) % len(data)  for i in ix]
  sents1 = [data[i][0] for i in ix]
  sents2 = [data[i][1] for i in next_ix]
  x = tokenizer(sents1, sents2, return_tensors="pt", padding=True, truncation=True)

  token_ids = torch.LongTensor(x["input_ids"])
  attention_mask = torch.LongTensor(x["attention_mask"])
  cls = token_ids == tokenizer.cls_token_id
  sep = token_ids == tokenizer.sep_token_id
  special = cls | sep
  mask = torch.rand(token_ids.shape) < 0.15
  mask = mask & (attention_mask == 1) & ~special
  mask_mask = (torch.rand(token_ids.shape) < 0.8) & mask
  mask_rand = (torch.rand(token_ids.shape) < 0.5) & mask & ~mask_mask

  y = token_ids.detach().clone()
  y[~mask] = -1

  token_ids[mask_mask] = tokenizer.mask_token_id
  token_ids[mask_rand] = torch.randint(
      0, tokenizer.vocab_size, token_ids[mask_rand].shape
  )

  nsp_target = ix == torch.tensor(next_ix)
  return x, y, nsp_target.float().unsqueeze(-1)

In [19]:
xb, yb, nsp_targets = get_batch('train')
tokenizer.decode(xb['input_ids'][0]), tokenizer.decode(yb[0]), yb.shape
print(nsp_targets)

input = [[tokenizer.decode([id]) for id in x] for x in xb['input_ids']]
target = [[tokenizer.decode([id]) for id in x] for x in yb]
df = pd.DataFrame({"input": input[0], "target": target[0]})
df.T
# for t1, t2 in zip(xb['input_ids'][0], yb[0]):
#   if t2 == -1:
#     print(tokenizer.decode([t1]))
#   else:
#     print(f'{tokenizer.decode([t1])} => {tokenizer.decode([t2])}')





tensor([[0.],
        [1.],
        [1.],
        [1.]])


,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
input,[CLS],[MASK],",",sc,(,fox,carolina,),-,spartan,...,malaysia,complied,with,orders,[MASK],drop,the,weapons,.,[SEP]
target,[UNK],moore,[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],-,[UNK],...,but,[UNK],[UNK],[UNK],to,[UNK],[UNK],[UNK],.,[UNK]


### Simple Model

In [20]:
import torch.nn as nn
from torch.nn import functional as F

class SimpleModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.word_embedding_table = nn.Embedding(vocab_size, 64)
    self.lm_head = nn.Linear(64, vocab_size, bias=False)
    self.lm_head.weight = self.word_embedding_table.weight
    self.mask_token_id = tokenizer.mask_token_id

  def forward(self, x, targets=None):
    embedding = self.word_embedding_table(x['input_ids']) # (B, T, C)
    logits = self.lm_head(embedding)

    if targets is None:
      return logits
    else:
      return logits, F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)

  def predict(self, x):
    idx = x['input_ids'] # (B, T)
    x['input_ids'] = torch.concat((torch.zeros(idx.shape[0], 1), idx[:, :-1]), dim=1).long()
    logits = self(x) # (B, T, C)
    mask = idx == self.mask_token_id # (B, T)
    probs = F.softmax(logits[mask], dim=-1) # (T_masked, C)
    # idx_masked = torch.multinomial(probs, num_samples=1) # (B, T_masked)
    top_probs = torch.topk(probs, 3, dim=-1) # (T_masked, 3)
    res = []
    for i in range(3):
      idx = idx.detach().clone() # (B, T)
      idx[mask] = top_probs.indices[:, i].view(-1)
      res.append(idx)
    res = torch.stack(res)
    res = res.permute(1, 0, 2)

    return res # (3, B, T)
    #  want (B, 3, T)

model = SimpleModel(tokenizer.vocab_size)

In [21]:
logits, loss = model(xb, yb)
print(logits.shape, loss.shape)
print(loss)


torch.Size([4, 68, 30522]) torch.Size([])
tensor(74.4705, grad_fn=<NllLossBackward0>)


In [22]:
x = ["I'm Afraid to Tell [MASK] Male Bosses I'm [MASK]",
     "The truth is that Tiger has been a nicer [MASK] than he has received credit for."]
x = tokenizer(x, return_tensors="pt", padding=True, truncation=True)
pred = model.predict(x)
pred.shape, tokenizer.batch_decode(pred[0]), tokenizer.batch_decode(pred[1])

(torch.Size([2, 3, 20]),
 ["[CLS] i'm afraid to tell tell male bosses i'm m [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]",
  "[CLS] i'm afraid to tell authentication male bosses i'm insider [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]",
  "[CLS] i'm afraid to tell doin male bosses i'mang [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]"],
 ['[CLS] the truth is that tiger has been a nicerr than he has received credit for. [SEP]',
  '[CLS] the truth is that tiger has been a nicer gradual than he has received credit for. [SEP]',
  '[CLS] the truth is that tiger has been a nicer waste than he has received credit for. [SEP]'])

In [23]:
x = ["I'm Afraid to Tell [MASK] Male Bosses I'm [MASK]", "I'm Afraid to Tell [MASK] Male Bosses I'm [MASK]"]
x = tokenizer(x, return_tensors="pt", padding=True, truncation=True)
idx = x['input_ids']
mask = idx == tokenizer.mask_token_id
# print(mask.shape)
logits = torch.randn(2, 15, 4)
# print(logits[mask].shape)
probs = F.softmax(logits[mask], dim=-1) # (T_masked, C)
top_probs = torch.topk(probs, 3, dim=-1) #(T_masked, C)
# print(idx[mask].shape)
print(top_probs.indices)
idx[mask] = top_probs.indices[:, 0].view(-1)
print(top_probs.indices[:, 0].view(-1))
idx
# idx[mask].shape
# pred = model.predict(x)
# tokenizer.batch_decode(pred)

tensor([[3, 0, 1],
        [3, 0, 2],
        [3, 1, 2],
        [3, 0, 1]])
tensor([3, 3, 3, 3])


tensor([[  101,  1045,  1005,  1049,  4452,  2000,  2425,     3,  3287, 23029,
          1045,  1005,  1049,     3,   102],
        [  101,  1045,  1005,  1049,  4452,  2000,  2425,     3,  3287, 23029,
          1045,  1005,  1049,     3,   102]])

### Attention

In [24]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

logits = torch.randn((B, T, T))
attn = F.softmax(logits, dim=-1)
print(attn.sum(dim=-1)) # probs sum to 1 for each token/row
res = attn @ x
attn.shape, res.shape

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])


(torch.Size([4, 8, 8]), torch.Size([4, 8, 32]))

In [25]:
# for self-attention, logits come from x
logits = x @ x.transpose(-1, -2) # (B, T, C) @ (B, C, T) ---> (B, T, T)

logits2 = []
for i in range(B):
  logits_b = []
  for j in range(T):
    logits_b.append(x[i, j] @ x[i].T) # (, C) @ (C, T) ---> (, T)

  logits2.append(torch.stack(logits_b)) # (T, T)
logits2 = torch.stack(logits2)

torch.allclose(logits, logits2)

True

In [26]:
import torch.nn as nn

key = nn.Linear(C, C, bias=False)
query = nn.Linear(C, C, bias=False)
value = nn.Linear(C, C, bias=False)
q = query(x)
k = key(x)
v = value(x)

logits = q @ k.transpose(-1, -2) # (B, T, C) @ (B, C, T) ---> (B, T, T)
attn = F.softmax(logits, dim=-1)
res = attn @ v # (B, T, T) @ (B, T, C)
res.shape

torch.Size([4, 8, 32])

Multi-head

In [27]:
H = 2
C_H = C // H
q_h = q.view(B, T, H, -1).transpose(1, 2)
k_h = k.view(B, T, H, -1).transpose(1, 2)
# logits = torch.randn((B, H, T, T))
logits = q_h @ k_h.transpose(-1, -2) # (B, H, T, C_H) @ (B, H, C_H, T) ---> (B, H, T, T)
attn = F.softmax(logits, dim=-1)
v_h = v.view(B, T, H, -1).transpose(1, 2)
res = attn @ v_h # (B, H, T, T) @ (B, H, T, C/H) ---> (B, H, T, C/H)
res.shape
res.transpose(1, 2).reshape(B, T, -1).shape

torch.Size([4, 8, 32])

Scaled: formular Var(aX), Var(X+Y) and Var(XY) in https://en.wikipedia.org/wiki/Variance

In [28]:
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1))
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1))

# Increasing the variance of the logits makes the probs moving to more peaky,
# coverging to one hot

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [29]:
k = torch.randn(100)
q = torch.randn(100)
prod = k * q
print(k.var(), q.var(), prod.var()) # var(XY) = 1

k = torch.randn(T, 100)
q = torch.randn(T, 100)
logits = q @ k.T
logits.var() # var(X+Y) = 2*var(X)

tensor(0.9592) tensor(0.9445) tensor(0.8309)


tensor(93.2931)

In [30]:
k = torch.randn(B,T,C_H)
q = torch.randn(B,T,C_H)
# var(k @ q.T) = C_H * var(k)
# var(k @ q.T * x) = C_H * var(k) * x^2
# x^2 = 1/C_H
wei = q @ k.transpose(-2, -1) * C_H**-0.5
wei.var()

tensor(1.2408)

### Bert Model

In [31]:
# BertModel
#   word_embedding, pos_emdding, tk_type_emdding
#   BertLayer *
#     SelfAttention
#     mlp
#   cls_head
#   lm_head

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

hidden_size = 64
num_head = 4
drop_out = 0.0
num_layers = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_iters = 60000
eval_interval = 100
lr = 2e-5
eval_iters = 200
batch_size = 16



class SelfAttention(nn.Module):
  def __init__(self):
    super().__init__()
    self.head_size = hidden_size // num_head

    self.query = nn.Linear(hidden_size, hidden_size)
    self.key = nn.Linear(hidden_size, hidden_size)
    self.value = nn.Linear(hidden_size, hidden_size)

    self.dense = nn.Linear(hidden_size, hidden_size)
    self.dropout = nn.Dropout(drop_out)

  def forward(self, x, mask):
    B, T, C = x.shape
    k = self.key(x)
    k = k.view(B, T, num_head, -1).transpose(1, 2) #(B, num_head, T, head_size)
    v = self.value(x)
    v = v.view(B, T, num_head, -1).transpose(1, 2) #(B, num_head, T, head_size)
    q = self.query(x)
    q = q.view(B, T, num_head, -1).transpose(1, 2) #(B, num_head, T, head_size)

    logits = q @ k.transpose(-1, -2) / self.head_size ** 0.5
    mask = mask[:, None, None, :] # (bs, 1, 1, seq_len)
    logits = logits.masked_fill(mask == 0, float('-inf'))
    probs = F.softmax(logits, dim=-1)
    probs = self.dropout(probs)
    res = probs @ v # (B, num_head, T, head_size)
    res = res.transpose(1, 2).reshape(B, T, -1) # (B, T, C)
    return self.dropout(self.dense(res)) # (B, T, C)


class BertLayer(nn.Module):
  def __init__(self):
    super().__init__()
    self.sa = SelfAttention()
    self.ffwd = nn.Sequential(
        nn.Linear(hidden_size, 4 * hidden_size),
        nn.ReLU(),
        nn.Linear(4 * hidden_size, hidden_size),
        nn.Dropout(drop_out),
    )
    self.ln1 = nn.LayerNorm(hidden_size)
    self.ln2 = nn.LayerNorm(hidden_size)

  def forward(self, x, mask):
    x = x + self.sa(self.ln1(x), mask)
    x = x + self.ffwd(self.ln2(x))
    return x


class BertModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.word_embedding = nn.Embedding(tokenizer.vocab_size, hidden_size, padding_idx=tokenizer.pad_token_id)
    self.pos_embedding = nn.Embedding(tokenizer.model_max_length, hidden_size)
    self.tk_type_embedding = nn.Embedding(2, hidden_size)
    position_ids = torch.arange(tokenizer.model_max_length).unsqueeze(0)
    self.register_buffer("position_ids", position_ids)
    self.dropout = nn.Dropout(drop_out)
    self.ln = nn.LayerNorm(hidden_size)

    self.bert_layers = nn.ModuleList([BertLayer() for _ in range(num_layers)])

    self.cls_ln = nn.LayerNorm(hidden_size)
    self.cls_dense = nn.Linear(hidden_size, hidden_size)
    self.cls_af = nn.Tanh()
    self.cls_head = nn.Linear(hidden_size, 2)

    self.lm_ln = nn.LayerNorm(hidden_size)
    self.lm_head = nn.Linear(hidden_size, tokenizer.vocab_size, bias=False)
    self.lm_head.weight = self.word_embedding.weight

  def forward(self, encoding, cls_targets=None, lm_targets=None):
    ids = encoding['input_ids'] # (B, T)
    type_ids = encoding['token_type_ids']
    atten_mask = encoding['attention_mask']

    B, T = ids.shape
    word_embedding = self.word_embedding(ids)
    pos_ids = self.position_ids[:, :T]
    pos_embedding = self.pos_embedding(pos_ids)
    tk_type_embedding = self.tk_type_embedding(type_ids)
    embedding = word_embedding + pos_embedding + tk_type_embedding
    embedding = self.dropout(self.ln(embedding))

    for bert_layer in self.bert_layers:
      embedding = bert_layer(embedding, atten_mask)

    cls_logit = self.cls_dense(embedding[:, 0])
    cls_logit = self.cls_head(self.cls_af(cls_logit))
    lm_logits = self.lm_head(embedding)

    cls_loss = None
    if cls_targets is not None:
      cls_loss = F.cross_entropy(cls_logit, cls_targets)

    lm_loss = None
    if lm_targets is not None:
      lm_loss = F.cross_entropy(lm_logits.view(-1, lm_logits.size(-1)),
                                lm_targets.view(-1),
                                ignore_index=-1)

    return cls_logit, lm_logits, cls_loss, lm_loss

  def predict(self, encoding):
    ids = encoding['input_ids'] # (B, T)

    # x['input_ids'] = torch.concat((torch.zeros(idx.shape[0], 1), idx[:, :-1]), dim=1).long()
    cls_logit, lm_logits, _, _ = self(encoding) # (B, 1) (B, T, C)
    mask = ids == tokenizer.mask_token_id # (B, T)
    probs = F.softmax(lm_logits[mask], dim=-1) # (T_masked, C)
    # idx_masked = torch.multinomial(probs, num_samples=1) # (B, T_masked)
    top_probs = torch.topk(probs, 3, dim=-1) # (T_masked, 3)
    res = []
    for i in range(3):
      ids = ids.detach().clone() # (B, T)
      ids[mask] = top_probs.indices[:, i].view(-1)
      res.append(ids)
    res = torch.stack(res) # (3, B, T
    res = res.permute(1, 0, 2) #  (B, 3, T)

    nsp = F.softmax(cls_logit, dim=-1)

    return res, nsp



In [ ]:
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data), (batch_size,))
  next_ix =[i if random.random() < 0.5 else (i+1) % len(data)  for i in ix]
  sents1 = [data[i][0] for i in ix]
  sents2 = [data[i][1] for i in next_ix]
  x = tokenizer(sents1, sents2, return_tensors="pt", padding=True, truncation=True)

  token_ids = torch.LongTensor(x["input_ids"])
  attention_mask = torch.LongTensor(x["attention_mask"])
  cls = token_ids == tokenizer.cls_token_id
  sep = token_ids == tokenizer.sep_token_id
  special = cls | sep
  mask = torch.rand(token_ids.shape) < 0.15
  mask = mask & (attention_mask == 1) & ~special
  mask_mask = (torch.rand(token_ids.shape) < 0.8) & mask
  mask_rand = (torch.rand(token_ids.shape) < 0.5) & mask & ~mask_mask

  y = token_ids.detach().clone()
  y[~mask] = -1

  token_ids[mask_mask] = tokenizer.mask_token_id
  token_ids[mask_rand] = torch.randint(
      0, tokenizer.vocab_size, token_ids[mask_rand].shape
  )

  nsp_target = ix == torch.tensor(next_ix)
  return x.to(device), y.to(device), nsp_target.long().to(device)


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        nsp_losses = torch.zeros(eval_iters)
        lm_losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y, nsp_targets = get_batch(split)
            _, _, nsp_loss, lm_loss = model(X, nsp_targets, Y)
            nsp_losses[k] = nsp_loss.item()
            lm_losses[k] = lm_loss.item()
        out[split] = nsp_losses.mean(), lm_losses.mean()
    model.train()
    return out

model = BertModel()
model = model.to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
# for n, p in model.named_parameters():
#   print(n, p.numel()/1e6, 'M')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss nsp={losses['train'][0]:.4f} lm={losses['train'][1]:.4f}, val loss nsp={losses['val'][0]:.4f} lm={losses['val'][1]:.4f}")

    # sample a batch of data
    xb, yb, nsp_targets = get_batch('train')

    # evaluate the loss
    _, _, nsp_loss, lm_loss = model(xb, nsp_targets, yb)
    optimizer.zero_grad(set_to_none=True)
    loss = nsp_loss + lm_loss
    loss.backward()
    optimizer.step()

# generate from the model
# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


2.190914 M parameters
step 0: train loss nsp=0.7276 lm=34.4385, val loss nsp=0.7333 lm=34.2982
step 100: train loss nsp=0.6924 lm=25.3994, val loss nsp=0.6936 lm=25.2682
step 200: train loss nsp=0.6931 lm=21.7050, val loss nsp=0.6922 lm=21.6340
step 300: train loss nsp=0.6935 lm=19.7910, val loss nsp=0.6928 lm=19.8130
step 400: train loss nsp=0.6936 lm=18.4664, val loss nsp=0.6936 lm=18.5642
step 500: train loss nsp=0.6930 lm=17.2938, val loss nsp=0.6933 lm=17.3404
step 600: train loss nsp=0.6930 lm=16.3680, val loss nsp=0.6931 lm=16.5312
step 700: train loss nsp=0.6936 lm=15.6111, val loss nsp=0.6954 lm=15.5438
step 800: train loss nsp=0.6927 lm=14.8044, val loss nsp=0.6928 lm=14.7703
step 900: train loss nsp=0.6934 lm=14.2180, val loss nsp=0.6937 lm=14.2012
step 1000: train loss nsp=0.6926 lm=13.6505, val loss nsp=0.6928 lm=13.5930
step 1100: train loss nsp=0.6929 lm=13.0999, val loss nsp=0.6931 lm=13.0416
step 1200: train loss nsp=0.6927 lm=12.6610, val loss nsp=0.6932 lm=12.6231
st

In [49]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

xb, yb, nsp_targets = get_batch('dev')
model(xb, lm_targets=yb)
pred, nsp = model.predict(xb)

print(xb['input_ids'].shape, yb.shape, pred.shape)
input = [[tokenizer.decode([id]) for id in x] for x in xb['input_ids']]
target = [[tokenizer.decode([id]) if id != -1 else "" for id in y] for y in yb]
pred = [[[tokenizer.decode([id]) if yid != -1 else "" for id, yid in zip(p, y)] for p in p3] for p3, y in zip(pred, yb)]
len(input), len(target), len(pred)
df = pd.DataFrame({'input': input[0], 'target': target[0],
                   'pred1': pred[0][0],
                   'pred2': pred[0][1],
                   'pred3': pred[0][2]})
print(nsp)
print(nsp_targets)
df.T


torch.Size([16, 74]) torch.Size([16, 74]) torch.Size([16, 3, 74])
tensor([[0.5037, 0.4963],
        [0.4786, 0.5214],
        [0.5110, 0.4890],
        [0.5071, 0.4929],
        [0.5017, 0.4983],
        [0.5003, 0.4997],
        [0.4770, 0.5230],
        [0.5069, 0.4931],
        [0.5062, 0.4938],
        [0.4982, 0.5018],
        [0.5057, 0.4943],
        [0.4840, 0.5160],
        [0.4955, 0.5045],
        [0.5087, 0.4913],
        [0.5131, 0.4869],
        [0.4817, 0.5183]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1], device='cuda:0')


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73
input,[CLS],however,",",weekend,shut,##down,##s,on,the,red,line,[MASK],next,weekend,.,[SEP],the,mb,##ta,planned,shut,##down,##s,of,weekend,service,to,the,orange,line,as,part,of,[MASK],five,[MASK],year,",",$,8,billion,capital,##bird,program,.,[SEP],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD]
target,,,,,,,,,,,,begin,,,,,,,,,,,,,,,,,orange,,,,,its,,-,,,,,,,investment,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
pred1,,,,,,,,,,,,the,,,,,,,,,,,,,,,,,orange,,,,,the,,the,,,,,,,##bird,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
pred2,,,,,,,,,,,,",",,,,,,,,,,,,,,,,,orange,,,,,",",,",",,,,,,,##bird,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
pred3,,,,,,,,,,,,a,,,,,,,,,,,,,,,,,orange,,,,,to,,to,,,,,,,##bird,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
model(xb, lm_targets=yb, cls_targets=nsp_targets)